<a href="https://colab.research.google.com/github/jakeoung/ShapeFromProjections/blob/master/ctdr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install shapefromprojections package
!git clone https://github.com/jakeoung/shapefromprojections
%cd shapefromprojections
!pip install -e .

fatal: destination path 'shapefromprojections' already exists and is not an empty directory.
/content/shapefromprojections
Obtaining file:///content/shapefromprojections
  Found existing installation: ctdrm 0.9.0
    Can't uninstall 'ctdrm'. No files were found to uninstall.
  Running setup.py develop for ctdrm


In [2]:
# install CUDA kernels
%cd ctdr/cuda
!python build.py build_ext --inplace
%cd ../../run

/content/shapefromprojections/ctdr/cuda
running build_ext
/usr/local/lib/python3.6/dist-packages/torch/utils/cpp_extension.py:335: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
copying build/lib.linux-x86_64-3.6/cuda_diff_fp.cpython-36m-x86_64-linux-gnu.so -> 
/content/shapefromprojections/run


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import h5py
import time

import ctdr
from parse_args import args, update_args
from ctdr.model.vanilla import Model
from ctdr.dataset import init_mesh
from ctdr.utils import util_mesh
from ctdr import optimize
import subprocess

#torch.backends.cudnn.benchmark=True

#------------------------------------------------
# load data
#------------------------------------------------
from ctdr.dataset import dataset

args.data='2starA'
# args.niter=3000
update_args(args)

if args.data.find("tomop") > 0:
    args.nmaterials = int(args.data[-3:-1])+1

ds = dataset.SinoDataset(args.ddata, args.nmaterials, args.eta)
width_physical = ds.proj_geom['DetectorSpacingX']*ds.proj_geom['DetectorColCount']
height_physical = ds.proj_geom['DetectorSpacingY']*ds.proj_geom['DetectorRowCount']
physical_unit = min(width_physical, height_physical)

finit_obj = args.ddata+'/init.obj'

# if os.path.exists(finit_obj) == False:
if True:
    init_mesh.save_init_mesh(finit_obj, args.data, args.nmaterials, physical_unit, args.subdiv)
else:
    print(f"Use existing init file {finit_obj}")

use_center_param = False
mus = np.arange(ds.nmaterials) / (ds.nmaterials-1)

print(finit_obj)
# refine
model = Model(finit_obj, ds.proj_geom, args.nmaterials,
              mus, args.nmu0, wlap=args.wlap, wflat=args.wflat).cuda()

Namespace(b=0, cuda=-1, data='2starA', dataroot='../data/', ddata='../data/2starA/', dresult='../result/2starA/ours_-b_0_-eta_0_-lr_0.01_-niter_500_-niter0_0_-nmu0_1_-subdiv_4_-wedge_2.0_-wflat_0.01_-wlap_10.0_/', eta=0, lr=0.01, niter=500, niter0=0, nmaterials=2, nmu0=1, resroot='../result', subdiv=4, verbose=1, wedge=2.0, wflat=0.01, wlap=10.0)


In [ ]:
phat = optimize.run(model, ds, args.niter, args, args.niter0)

# util_mesh.save_mesh(args.dresult+f'{epoch:04d}.obj', vv.numpy(), ff.numpy(), labels_v, labels_f)

#------------------------------------------------
# save output
#------------------------------------------------
hf = h5py.File(args.dresult+"res.h5", "w")
hf.close()

../data/2starA//init.obj
@statistics of mesh: # of v: 2562, f: 5120
set mu as parameters
initialize flatten loss
@ model.mus Parameter containing:
tensor([0., 1.], device='cuda:0', requires_grad=True)
@ statistics of mesh: 2562, 5120
~ 000 l2_loss: 0.09115827 edge: 0.000458 lap: 0.000003 flat: 0.000975 mus: [0.01 1.01] time: 0.8415

tensor(0., device='cuda:0', grad_fn=<MinBackward1>) tensor(0.8000, device='cuda:0', grad_fn=<MaxBackward1>)
ndead nneg phat.min and max_v: 1 1 -0.00, 1.50
ndead nneg phat.min and max_v: 5 5 -0.07, 1.50
ndead nneg phat.min and max_v: 5 5 -0.06, 1.51
ndead nneg phat.min and max_v: 3 3 -0.03, 1.51
ndead nneg phat.min and max_v: 22 22 -0.08, 1.52
ndead nneg phat.min and max_v: 7 7 -0.08, 1.52
ndead nneg phat.min and max_v: 12 12 -0.05, 1.53
ndead nneg phat.min and max_v: 8 8 -0.02, 1.53
ndead nneg phat.min and max_v: 6 6 -0.03, 1.54
ndead nneg phat.min and max_v: 3 3 -0.03, 1.55
ndead nneg phat.min and max_v: 1 1 -3.76, 1.62
@ statistics of mesh: 2562, 5120
~ 0